In [12]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models.models import LstmMseDropout
from utils.data_loader import DataPreperatorPrediction, DataSet
from loss_module import LossModuleMse, LossModuleMle
from utils.predictor import Predictor

In [13]:
param = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../data/artifical_signals/NewBlade_random_walk_time_difference.csv',
        "droped_feature" : ["timestamp"]
    },
    "model" : {
        "path" : "../../models/MSE_model/random_walk_time_difference_InputSize7_LayerLstm2_HiddenLstm200_HiddenFc75_Seq100.pt",
        "input_size" : 7,
        "n_hidden_lstm" : 200,
        "sequence_size" : 100,
        "batch_size" : 1,  # Has to be 1 in prediction mode!!!
        "lstm_layer" : 2,
        "n_hidden_fc": 75,
        "dropout_rate": 0.2
    },
    "results": {
        "path_prediction" : "../visualisation/files/prediction/random_walk_difference.csv",
        "path_residual" : "../visualisation/files/residuals/random_walk_difference.csv"
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from NewBlade Training Data

In [3]:
mean_training_data = [-5.37536613e-02, -2.53111489e-04, -8.82854465e+05, 7.79034183e+02, 1.45531178e+04, 1.37766733e+03, 6.50149764e-01] 
var_training_data = [1.25303578e-01, 1.16898690e-03, 2.86060835e+06, 1.64515717e+06, 6.85728371e+06, 3.63196175e+05, 8.21463343e-03]

### Mean and Variance from Artifical Training Data

In [4]:
mean_training_data= [-5.31764899e-02, -3.98576146e-04, -8.82773455e+05,  8.25672897e+02, 1.47034247e+04,  1.42685595e+03,  6.62155736e-01,  1.23172374e-02]
var_training_data = [1.28792583e-01, 1.21258617e-03, 2.90245238e+06, 1.72279458e+06, 6.83095901e+06, 3.12357562e+05, 3.89033076e-03, 5.01164766e+01]

### Mean and Variance from Random Walk Data

In [5]:
mean_training_data =[-5.31764899e-02, -3.98576146e-04,  1.55255733e+03,  8.25672897e+02, 1.47034247e+04,  1.42685595e+03,  6.62155736e-01]
var_training_data =[1.28792583e-01, 1.21258617e-03, 2.04108811e+06, 1.72279458e+06, 6.83095901e+06, 3.12357562e+05, 3.89033076e-03]

### Mean and Variance from Random Walk Data with first order difference

In [14]:
mean_training_data =[-5.30145478e-02, -3.98852220e-04,  1.57238159e+00,  8.26196229e+02, 1.47058099e+04,  1.42778443e+03,  6.62286323e-01]
var_training_data =[1.28839165e-01, 1.21339499e-03, 1.10982448e+03, 1.72353307e+06, 6.82698220e+06, 3.11272820e+05, 3.86734667e-03]

## Create DataLoader

In [15]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=1, 
                         shuffle=False, 
                         drop_last=True)

   timestamp   cut torque   cut lag error   cut position   cut speed  \
0      0.008    -0.283251       -0.006468            0.0   48.065186   
1      0.012    -0.308639        0.000107            0.0   44.631958   
2      0.016    -0.331372        0.004387            1.0   51.498413   
3      0.020    -0.346761        0.004063            0.0   65.231323   
4      0.024    -0.297286       -0.014619            0.0   72.097778   

    film position   film speed   film lag error  
0           11128    32.556355         0.491569  
1           11128    40.069519         0.519276  
2           11129    45.078098         0.545538  
3           11129    52.591114         0.561331  
4           11129    57.599842         0.574634  
    cut torque   cut lag error   cut position   cut speed   film position  \
0    -0.283251       -0.006468            0.0   48.065186           11128   
1    -0.308639        0.000107            0.0   44.631958           11128   
2    -0.331372        0.004387      

## Define Model and load Parameters of trained model

In [16]:
model = LstmMseDropout(batch_size=param['model']['batch_size'], 
                       input_dim=param['model']['input_size'], 
                       n_hidden_lstm=param['model']['n_hidden_lstm'], 
                       n_hidden_fc=param['model']['n_hidden_fc'], 
                       n_layers=param['model']['lstm_layer'], 
                       dropout_rate= param['model']['dropout_rate'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Define Loss Function

In [17]:
criterion = LossModuleMse(param["model"]["input_size"], param["model"]["batch_size"])

## Initialize Trainer

In [18]:
predictor = Predictor(model=model,
                      criterion=criterion,
                      path_data=param["data"]["path"],
                      path_results=param["results"]["path_prediction"],
                      path_residuals=param["results"]["path_residual"],
                      columns_to_ignore=param["data"]["droped_feature"]
                      )

## Predict

In [19]:
results = predictor.predict(data_loader)
predictor.save_prediction(results)

Start predicting
Finished predicting


## Compute Residuals for New Blade

In [11]:
residuals = predictor.compute_residuals(results)
predictor.save_residuals(residuals)